In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from keras.constraints import maxnorm
from keras.models import model_from_json
from keras.models import load_model

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from skopt import BayesSearchCV

from sklearn import preprocessing
from sklearn import pipeline

import time
import joblib

from joblib import dump

In [2]:
X_treinamento = pd.read_json(r'C:\Users\Everton\Documents\Mestrado\projeto-final\dados\precipitacao\X_treinamento.json')
X_teste = pd.read_json      (r'C:\Users\Everton\Documents\Mestrado\projeto-final\dados\precipitacao\X_teste.json')
treino_json = pd.read_json  (r'C:\Users\Everton\Documents\Mestrado\projeto-final\dados\precipitacao\y_treinamento.json', typ='series')
teste_json = pd.read_json   (r'C:\Users\Everton\Documents\Mestrado\projeto-final\dados\precipitacao\y_teste.json', typ='series')

In [3]:
X_treinamento = X_treinamento.drop(['horario_experimento', 'mes_experimento', 'vazao_inicial', 'vazao_final', 'vazao_total'], axis=1)

In [4]:
X_teste = X_teste.drop(['horario_experimento', 'mes_experimento', 'vazao_inicial', 'vazao_final', 'vazao_total'], axis=1)

In [5]:
a = []
b = []

In [6]:
for i in range(treino_json.size):
    a.append(list(treino_json[i]))

In [7]:
for i in range(teste_json.size):
    b.append(list(teste_json[i]))

In [8]:
y_treinamento = np.array(a, dtype='float32')

In [9]:
y_teste = np.array(b, dtype='float32')

In [10]:
def build_model(
    optimizer,
    init_mode,
    activation,
    dropout_rate,
    weight_constraint,
    neurons,
    loss,
    regularizers
):
    model = models.Sequential()
    
    # layer 1
    model.add(
        layers.Dense(
            neurons,
            activation = activation,
            kernel_initializer=init_mode,
            kernel_constraint=maxnorm(weight_constraint),
            kernel_regularizer=regularizers,
            input_shape=[len(X_treinamento.keys())]
        )
    )
    model.add(layers.Dropout(dropout_rate))
    
    # layer 2
    model.add(layers.Dense(256))
    
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['mae', 'mse', keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [11]:
batch_size = [6, 9, 18]
epochs = [400, 500, 600, 700]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
neurons = [256, 512, 1024]
losses = ['mse', 'logcosh', 'huber']
regularizers = [tf.keras.regularizers.l1(0.0001), tf.keras.regularizers.l2(0.0001), tf.keras.regularizers.l1_l2(0.0001)]

In [12]:
scalers = [
    preprocessing.StandardScaler(),
    preprocessing.MinMaxScaler(),
    preprocessing.MaxAbsScaler(),
    preprocessing.RobustScaler(quantile_range=(25,75)),
    preprocessing.PowerTransformer(method='yeo-johnson'),
    preprocessing.QuantileTransformer(output_distribution='uniform'),
    preprocessing.QuantileTransformer(output_distribution='normal'),
]

In [13]:
param_grid = {
    'estimator__batch_size': batch_size,
    'estimator__epochs': epochs,
    'estimator__optimizer': optimizer,
    'estimator__init_mode': init_mode,
    'estimator__activation': activation,
    'estimator__weight_constraint': weight_constraint,
    'estimator__dropout_rate': dropout_rate,
    'estimator__neurons': neurons,
    'estimator__loss': losses,
    'estimator__regularizers': regularizers,
    'scaler': scalers
}

In [14]:
model = KerasRegressor(build_fn=build_model, verbose=1)

In [15]:
steps = [('scaler', preprocessing.StandardScaler()), 
         ('estimator', model)]

In [16]:
pipe = pipeline.Pipeline(steps)

In [17]:
pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('estimator',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001B336B46DF0>)])

In [18]:
grid = BayesSearchCV(
    estimator = pipe,
    search_spaces = param_grid,
    n_iter = 50,
    n_jobs = 2,
    cv = 10,
    scoring = 'neg_mean_squared_error',
    refit = True
)

In [19]:
start = time.time()
grid_result = grid.fit(X_treinamento, y_treinamento)
end = time.time()

C:\Users\Everton\anaconda3\envs\projeto-final\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/500
12/12 [==============================] - 0s 838us/step - loss: 2.5984 - mae: 3.0391 - mse: 19.9746 - root_mean_squared_error: 4.4693
Epoch 2/500
12/12 [==============================] - 0s 866us/step - loss: 2.5584 - mae: 2.9965 - mse: 19.5457 - root_mean_squared_error: 4.4211
Epoch 3/500
12/12 [==============================] - 0s 827us/step - loss: 2.4901 - mae: 2.9249 - mse: 18.8802 - root_mean_squared_error: 4.3451
Epoch 4/500
12/12 [==============================] - 0s 802us/step - loss: 2.3871 - mae: 2.8179 - mse: 17.7775 - root_mean_squared_error: 4.2163
Epoch 5/500
12/12 [==============================] - 0s 857us/step - loss: 2.2497 - mae: 2.6745 - mse: 16.3958 - root_mean_squared_error: 4.0492
Epoch 6/500
12/12 [==============================] - 0s 833us/step - loss: 2.0816 - mae: 2.5024 - mse: 14.6558 - root_mean_squared_error: 3.8283
Epoch 7/500
12/12 [==============================] - 0s 840us/step - loss: 1.8899 - mae: 2.3062 - mse: 12.7544 - root_mean_squared

12/12 [==============================] - 0s 821us/step - loss: 0.3847 - mae: 0.7095 - mse: 1.2830 - root_mean_squared_error: 1.1327
Epoch 58/500
12/12 [==============================] - 0s 847us/step - loss: 0.3799 - mae: 0.7053 - mse: 1.2597 - root_mean_squared_error: 1.1224
Epoch 59/500
12/12 [==============================] - 0s 831us/step - loss: 0.3795 - mae: 0.7025 - mse: 1.2695 - root_mean_squared_error: 1.1267
Epoch 60/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3806 - mae: 0.7041 - mse: 1.2619 - root_mean_squared_error: 1.1234
Epoch 61/500
12/12 [==============================] - 0s 818us/step - loss: 0.3831 - mae: 0.7089 - mse: 1.2766 - root_mean_squared_error: 1.1299
Epoch 62/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3722 - mae: 0.6928 - mse: 1.2405 - root_mean_squared_error: 1.1138
Epoch 63/500
12/12 [==============================] - 0s 837us/step - loss: 0.3792 - mae: 0.7034 - mse: 1.2557 - root_mean_squared_error: 1.1206
Ep

12/12 [==============================] - 0s 847us/step - loss: 0.3491 - mae: 0.6624 - mse: 1.1374 - root_mean_squared_error: 1.0665
Epoch 114/500
12/12 [==============================] - 0s 965us/step - loss: 0.3450 - mae: 0.6594 - mse: 1.1000 - root_mean_squared_error: 1.0488
Epoch 115/500
12/12 [==============================] - 0s 830us/step - loss: 0.3493 - mae: 0.6640 - mse: 1.1265 - root_mean_squared_error: 1.0614
Epoch 116/500
12/12 [==============================] - 0s 831us/step - loss: 0.3434 - mae: 0.6556 - mse: 1.1119 - root_mean_squared_error: 1.0545
Epoch 117/500
12/12 [==============================] - 0s 836us/step - loss: 0.3373 - mae: 0.6470 - mse: 1.0822 - root_mean_squared_error: 1.0403
Epoch 118/500
12/12 [==============================] - 0s 831us/step - loss: 0.3401 - mae: 0.6528 - mse: 1.0874 - root_mean_squared_error: 1.0428
Epoch 119/500
12/12 [==============================] - 0s 843us/step - loss: 0.3391 - mae: 0.6526 - mse: 1.0784 - root_mean_squared_error:

12/12 [==============================] - 0s 856us/step - loss: 0.2840 - mae: 0.5805 - mse: 0.8718 - root_mean_squared_error: 0.9337
Epoch 170/500
12/12 [==============================] - 0s 838us/step - loss: 0.2834 - mae: 0.5797 - mse: 0.8610 - root_mean_squared_error: 0.9279
Epoch 171/500
12/12 [==============================] - 0s 830us/step - loss: 0.2882 - mae: 0.5856 - mse: 0.8922 - root_mean_squared_error: 0.9446
Epoch 172/500
12/12 [==============================] - 0s 832us/step - loss: 0.2872 - mae: 0.5862 - mse: 0.8779 - root_mean_squared_error: 0.9370
Epoch 173/500
12/12 [==============================] - 0s 830us/step - loss: 0.2753 - mae: 0.5696 - mse: 0.8375 - root_mean_squared_error: 0.9152
Epoch 174/500
12/12 [==============================] - 0s 891us/step - loss: 0.2861 - mae: 0.5824 - mse: 0.8772 - root_mean_squared_error: 0.9366
Epoch 175/500
12/12 [==============================] - 0s 840us/step - loss: 0.2858 - mae: 0.5840 - mse: 0.8688 - root_mean_squared_error:

12/12 [==============================] - 0s 812us/step - loss: 0.2420 - mae: 0.5248 - mse: 0.7086 - root_mean_squared_error: 0.8418
Epoch 226/500
12/12 [==============================] - 0s 827us/step - loss: 0.2499 - mae: 0.5366 - mse: 0.7368 - root_mean_squared_error: 0.8584
Epoch 227/500
12/12 [==============================] - 0s 840us/step - loss: 0.2442 - mae: 0.5285 - mse: 0.7147 - root_mean_squared_error: 0.8454
Epoch 228/500
12/12 [==============================] - 0s 0s/step - loss: 0.2296 - mae: 0.5056 - mse: 0.6671 - root_mean_squared_error: 0.8168
Epoch 229/500
12/12 [==============================] - 0s 827us/step - loss: 0.2431 - mae: 0.5239 - mse: 0.7144 - root_mean_squared_error: 0.8452
Epoch 230/500
12/12 [==============================] - 0s 831us/step - loss: 0.2332 - mae: 0.5122 - mse: 0.6801 - root_mean_squared_error: 0.8247
Epoch 231/500
12/12 [==============================] - 0s 836us/step - loss: 0.2273 - mae: 0.5017 - mse: 0.6653 - root_mean_squared_error: 0.

12/12 [==============================] - 0s 833us/step - loss: 0.2242 - mae: 0.4987 - mse: 0.6399 - root_mean_squared_error: 0.7999
Epoch 282/500
12/12 [==============================] - 0s 843us/step - loss: 0.2191 - mae: 0.4913 - mse: 0.6252 - root_mean_squared_error: 0.7907
Epoch 283/500
12/12 [==============================] - 0s 825us/step - loss: 0.2184 - mae: 0.4898 - mse: 0.6307 - root_mean_squared_error: 0.7942
Epoch 284/500
12/12 [==============================] - 0s 831us/step - loss: 0.2258 - mae: 0.5005 - mse: 0.6589 - root_mean_squared_error: 0.8117
Epoch 285/500
12/12 [==============================] - 0s 841us/step - loss: 0.2183 - mae: 0.4912 - mse: 0.6287 - root_mean_squared_error: 0.7929
Epoch 286/500
12/12 [==============================] - 0s 833us/step - loss: 0.2157 - mae: 0.4882 - mse: 0.6218 - root_mean_squared_error: 0.7885
Epoch 287/500
12/12 [==============================] - 0s 832us/step - loss: 0.2224 - mae: 0.5000 - mse: 0.6329 - root_mean_squared_error:

12/12 [==============================] - 0s 833us/step - loss: 0.2085 - mae: 0.4782 - mse: 0.5894 - root_mean_squared_error: 0.7677
Epoch 338/500
12/12 [==============================] - 0s 825us/step - loss: 0.2074 - mae: 0.4785 - mse: 0.5871 - root_mean_squared_error: 0.7662
Epoch 339/500
12/12 [==============================] - 0s 826us/step - loss: 0.2099 - mae: 0.4751 - mse: 0.6035 - root_mean_squared_error: 0.7769
Epoch 340/500
12/12 [==============================] - 0s 833us/step - loss: 0.2178 - mae: 0.4919 - mse: 0.6172 - root_mean_squared_error: 0.7856
Epoch 341/500
12/12 [==============================] - 0s 793us/step - loss: 0.2145 - mae: 0.4822 - mse: 0.6211 - root_mean_squared_error: 0.7881
Epoch 342/500
12/12 [==============================] - 0s 862us/step - loss: 0.2035 - mae: 0.4712 - mse: 0.5717 - root_mean_squared_error: 0.7561
Epoch 343/500
12/12 [==============================] - 0s 836us/step - loss: 0.2195 - mae: 0.4928 - mse: 0.6229 - root_mean_squared_error:

12/12 [==============================] - 0s 837us/step - loss: 0.2022 - mae: 0.4665 - mse: 0.5683 - root_mean_squared_error: 0.7539
Epoch 394/500
12/12 [==============================] - 0s 839us/step - loss: 0.1979 - mae: 0.4595 - mse: 0.5551 - root_mean_squared_error: 0.7451
Epoch 395/500
12/12 [==============================] - 0s 830us/step - loss: 0.2080 - mae: 0.4784 - mse: 0.5822 - root_mean_squared_error: 0.7630
Epoch 396/500
12/12 [==============================] - 0s 802us/step - loss: 0.1978 - mae: 0.4633 - mse: 0.5539 - root_mean_squared_error: 0.7443
Epoch 397/500
12/12 [==============================] - 0s 864us/step - loss: 0.2109 - mae: 0.4792 - mse: 0.5991 - root_mean_squared_error: 0.7740
Epoch 398/500
12/12 [==============================] - 0s 839us/step - loss: 0.2054 - mae: 0.4703 - mse: 0.5851 - root_mean_squared_error: 0.7649
Epoch 399/500
12/12 [==============================] - 0s 845us/step - loss: 0.2049 - mae: 0.4758 - mse: 0.5715 - root_mean_squared_error:

12/12 [==============================] - 0s 823us/step - loss: 0.2008 - mae: 0.4606 - mse: 0.5693 - root_mean_squared_error: 0.7545
Epoch 450/500
12/12 [==============================] - 0s 804us/step - loss: 0.2016 - mae: 0.4687 - mse: 0.5586 - root_mean_squared_error: 0.7474
Epoch 451/500
12/12 [==============================] - 0s 844us/step - loss: 0.1973 - mae: 0.4605 - mse: 0.5538 - root_mean_squared_error: 0.7442
Epoch 452/500
12/12 [==============================] - 0s 821us/step - loss: 0.2180 - mae: 0.4869 - mse: 0.6195 - root_mean_squared_error: 0.7871
Epoch 453/500
12/12 [==============================] - 0s 843us/step - loss: 0.2000 - mae: 0.4662 - mse: 0.5660 - root_mean_squared_error: 0.7523
Epoch 454/500
12/12 [==============================] - 0s 831us/step - loss: 0.1987 - mae: 0.4601 - mse: 0.5561 - root_mean_squared_error: 0.7457
Epoch 455/500
12/12 [==============================] - 0s 834us/step - loss: 0.2004 - mae: 0.4660 - mse: 0.5582 - root_mean_squared_error:

In [20]:
print((end-start)/60)

53.57156726121902


In [21]:
grid_result.best_params_

OrderedDict([('estimator__activation', 'softsign'),
             ('estimator__batch_size', 6),
             ('estimator__dropout_rate', 0.1),
             ('estimator__epochs', 500),
             ('estimator__init_mode', 'normal'),
             ('estimator__loss', 'logcosh'),
             ('estimator__neurons', 256),
             ('estimator__optimizer', 'RMSprop'),
             ('estimator__regularizers',
              <tensorflow.python.keras.regularizers.L2 at 0x1b336e02340>),
             ('estimator__weight_constraint', 5),
             ('scaler', PowerTransformer())])

In [22]:
grid_result.best_index_

46

In [23]:
hist = pd.DataFrame(grid_result.best_estimator_['estimator'].model.history.history)

In [24]:
hist.to_json(r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\prec-1-10-history')

In [25]:
loaded_history = pd.read_json(r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\prec-1-10-history')

In [26]:
loaded_history

,loss,mae,mse,root_mean_squared_error
0,2.598410,3.039117,19.974560,4.469291
1,2.558392,2.996497,19.545700,4.421052
2,2.490104,2.924904,18.880178,4.345133
3,2.387051,2.817912,17.777464,4.216333
4,2.249675,2.674482,16.395805,4.049173
...,...,...,...,...
495,0.194095,0.458160,0.535912,0.732060
496,0.193403,0.456027,0.535374,0.731692
497,0.193462,0.451675,0.538359,0.733729
498,0.196524,0.463222,0.538456,0.733796


In [27]:
cv_results_ = pd.DataFrame(grid_result.cv_results_)

In [28]:
cv_results_.iloc[:, 0:17].to_json(r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\prec_1_10_cv_results_', orient='columns')

In [29]:
model = grid_result.best_estimator_['estimator'].model

In [30]:
model.save(r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\model-70-30-prec-1-10.h5')

In [31]:
dump(grid_result.best_estimator_['scaler'], r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\scaler-70-30-prec-1-10.joblib') 

['C:\\Users\\Everton\\Documents\\Mestrado\\projeto-final\\experimentos\\precipitacao-experimento-1\\resultados\\Experimento-10\\scaler-70-30-prec-1-10.joblib']

In [32]:
X_treinamento_100 = pd.concat([X_treinamento, X_teste], axis=0)
y_treinamento_100 = pd.concat([treino_json, teste_json], axis=0)

X_treinamento_100.reset_index(drop=True, inplace=True)
y_treinamento_100.reset_index(drop=True, inplace=True)

a100 = []

for i in range(y_treinamento_100.size):
    a100.append(list(y_treinamento_100[i]))
    
y_treinamento_100 = np.array(a100, dtype='float32')

In [33]:
grid_result.best_estimator_.fit(X_treinamento_100, y_treinamento_100)

Epoch 1/500
17/17 [==============================] - 0s 1ms/step - loss: 2.6850 - mae: 3.1305 - mse: 20.7778 - root_mean_squared_error: 4.5583
Epoch 2/500
17/17 [==============================] - 0s 919us/step - loss: 2.6039 - mae: 3.0454 - mse: 19.9097 - root_mean_squared_error: 4.4620
Epoch 3/500
17/17 [==============================] - 0s 919us/step - loss: 2.4491 - mae: 2.8829 - mse: 18.2506 - root_mean_squared_error: 4.2721
Epoch 4/500
17/17 [==============================] - 0s 919us/step - loss: 2.2259 - mae: 2.6530 - mse: 15.8994 - root_mean_squared_error: 3.9874
Epoch 5/500
17/17 [==============================] - 0s 919us/step - loss: 1.9552 - mae: 2.3757 - mse: 13.2285 - root_mean_squared_error: 3.6371
Epoch 6/500
17/17 [==============================] - 0s 919us/step - loss: 1.6487 - mae: 2.0610 - mse: 10.2731 - root_mean_squared_error: 3.2052
Epoch 7/500
17/17 [==============================] - 0s 383us/step - loss: 1.3347 - mae: 1.7372 - mse: 7.4964 - root_mean_squared_er

17/17 [==============================] - 0s 1ms/step - loss: 0.3558 - mae: 0.6767 - mse: 1.1393 - root_mean_squared_error: 1.0674
Epoch 58/500
17/17 [==============================] - 0s 920us/step - loss: 0.3521 - mae: 0.6728 - mse: 1.1173 - root_mean_squared_error: 1.0570
Epoch 59/500
17/17 [==============================] - 0s 919us/step - loss: 0.3577 - mae: 0.6793 - mse: 1.1427 - root_mean_squared_error: 1.0690
Epoch 60/500
17/17 [==============================] - 0s 919us/step - loss: 0.3496 - mae: 0.6697 - mse: 1.1081 - root_mean_squared_error: 1.0526
Epoch 61/500
17/17 [==============================] - 0s 919us/step - loss: 0.3542 - mae: 0.6749 - mse: 1.1335 - root_mean_squared_error: 1.0647
Epoch 62/500
17/17 [==============================] - 0s 1ms/step - loss: 0.3509 - mae: 0.6701 - mse: 1.1187 - root_mean_squared_error: 1.0577
Epoch 63/500
17/17 [==============================] - 0s 919us/step - loss: 0.3553 - mae: 0.6762 - mse: 1.1351 - root_mean_squared_error: 1.0654
Ep

17/17 [==============================] - 0s 919us/step - loss: 0.2976 - mae: 0.6021 - mse: 0.9069 - root_mean_squared_error: 0.9523
Epoch 114/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2965 - mae: 0.5977 - mse: 0.9069 - root_mean_squared_error: 0.9523
Epoch 115/500
17/17 [==============================] - 0s 919us/step - loss: 0.2958 - mae: 0.5993 - mse: 0.8961 - root_mean_squared_error: 0.9466
Epoch 116/500
17/17 [==============================] - 0s 919us/step - loss: 0.2946 - mae: 0.5965 - mse: 0.8953 - root_mean_squared_error: 0.9462
Epoch 117/500
17/17 [==============================] - 0s 919us/step - loss: 0.2859 - mae: 0.5857 - mse: 0.8613 - root_mean_squared_error: 0.9281
Epoch 118/500
17/17 [==============================] - 0s 919us/step - loss: 0.2885 - mae: 0.5894 - mse: 0.8748 - root_mean_squared_error: 0.9353
Epoch 119/500
17/17 [==============================] - 0s 919us/step - loss: 0.2871 - mae: 0.5882 - mse: 0.8624 - root_mean_squared_error: 0

17/17 [==============================] - 0s 919us/step - loss: 0.2378 - mae: 0.5207 - mse: 0.6869 - root_mean_squared_error: 0.8288
Epoch 169/500
17/17 [==============================] - 0s 919us/step - loss: 0.2410 - mae: 0.5267 - mse: 0.6890 - root_mean_squared_error: 0.8301
Epoch 170/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2443 - mae: 0.5287 - mse: 0.7047 - root_mean_squared_error: 0.8394
Epoch 171/500
17/17 [==============================] - 0s 920us/step - loss: 0.2408 - mae: 0.5228 - mse: 0.7013 - root_mean_squared_error: 0.8374
Epoch 172/500
17/17 [==============================] - 0s 919us/step - loss: 0.2382 - mae: 0.5207 - mse: 0.6826 - root_mean_squared_error: 0.8262
Epoch 173/500
17/17 [==============================] - 0s 920us/step - loss: 0.2352 - mae: 0.5183 - mse: 0.6731 - root_mean_squared_error: 0.8205
Epoch 174/500
17/17 [==============================] - 0s 919us/step - loss: 0.2445 - mae: 0.5302 - mse: 0.7051 - root_mean_squared_error: 0

17/17 [==============================] - 0s 919us/step - loss: 0.2256 - mae: 0.5033 - mse: 0.6389 - root_mean_squared_error: 0.7993
Epoch 225/500
17/17 [==============================] - 0s 919us/step - loss: 0.2288 - mae: 0.5094 - mse: 0.6468 - root_mean_squared_error: 0.8042
Epoch 226/500
17/17 [==============================] - 0s 919us/step - loss: 0.2236 - mae: 0.5012 - mse: 0.6300 - root_mean_squared_error: 0.7937
Epoch 227/500
17/17 [==============================] - 0s 383us/step - loss: 0.2323 - mae: 0.5140 - mse: 0.6639 - root_mean_squared_error: 0.8148
Epoch 228/500
17/17 [==============================] - 0s 919us/step - loss: 0.2196 - mae: 0.4947 - mse: 0.6216 - root_mean_squared_error: 0.7884
Epoch 229/500
17/17 [==============================] - 0s 919us/step - loss: 0.2251 - mae: 0.5021 - mse: 0.6375 - root_mean_squared_error: 0.7985
Epoch 230/500
17/17 [==============================] - 0s 919us/step - loss: 0.2218 - mae: 0.4991 - mse: 0.6248 - root_mean_squared_error:

17/17 [==============================] - 0s 919us/step - loss: 0.2185 - mae: 0.4948 - mse: 0.6123 - root_mean_squared_error: 0.7825
Epoch 281/500
17/17 [==============================] - 0s 383us/step - loss: 0.2161 - mae: 0.4910 - mse: 0.6031 - root_mean_squared_error: 0.7766
Epoch 282/500
17/17 [==============================] - 0s 920us/step - loss: 0.2216 - mae: 0.5001 - mse: 0.6174 - root_mean_squared_error: 0.7857
Epoch 283/500
17/17 [==============================] - 0s 919us/step - loss: 0.2246 - mae: 0.5016 - mse: 0.6361 - root_mean_squared_error: 0.7976
Epoch 284/500
17/17 [==============================] - 0s 919us/step - loss: 0.2166 - mae: 0.4901 - mse: 0.6059 - root_mean_squared_error: 0.7784
Epoch 285/500
17/17 [==============================] - 0s 919us/step - loss: 0.2133 - mae: 0.4889 - mse: 0.5925 - root_mean_squared_error: 0.7697
Epoch 286/500
17/17 [==============================] - 0s 919us/step - loss: 0.2106 - mae: 0.4825 - mse: 0.5837 - root_mean_squared_error:

17/17 [==============================] - 0s 383us/step - loss: 0.2108 - mae: 0.4831 - mse: 0.5827 - root_mean_squared_error: 0.7633
Epoch 337/500
17/17 [==============================] - 0s 919us/step - loss: 0.2058 - mae: 0.4747 - mse: 0.5711 - root_mean_squared_error: 0.7557
Epoch 338/500
17/17 [==============================] - 0s 919us/step - loss: 0.2075 - mae: 0.4782 - mse: 0.5731 - root_mean_squared_error: 0.7571
Epoch 339/500
17/17 [==============================] - 0s 919us/step - loss: 0.2198 - mae: 0.4975 - mse: 0.6056 - root_mean_squared_error: 0.7782
Epoch 340/500
17/17 [==============================] - 0s 919us/step - loss: 0.2146 - mae: 0.4862 - mse: 0.6021 - root_mean_squared_error: 0.7760
Epoch 341/500
17/17 [==============================] - 0s 919us/step - loss: 0.2019 - mae: 0.4671 - mse: 0.5606 - root_mean_squared_error: 0.7487
Epoch 342/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2078 - mae: 0.4809 - mse: 0.5728 - root_mean_squared_error: 0

17/17 [==============================] - 0s 920us/step - loss: 0.2018 - mae: 0.4685 - mse: 0.5545 - root_mean_squared_error: 0.7446
Epoch 393/500
17/17 [==============================] - 0s 919us/step - loss: 0.1990 - mae: 0.4677 - mse: 0.5373 - root_mean_squared_error: 0.7330
Epoch 394/500
17/17 [==============================] - 0s 919us/step - loss: 0.2085 - mae: 0.4800 - mse: 0.5749 - root_mean_squared_error: 0.7582
Epoch 395/500
17/17 [==============================] - 0s 919us/step - loss: 0.2023 - mae: 0.4677 - mse: 0.5587 - root_mean_squared_error: 0.7474
Epoch 396/500
17/17 [==============================] - 0s 919us/step - loss: 0.2100 - mae: 0.4813 - mse: 0.5796 - root_mean_squared_error: 0.7613
Epoch 397/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2132 - mae: 0.4823 - mse: 0.5951 - root_mean_squared_error: 0.7714
Epoch 398/500
17/17 [==============================] - 0s 920us/step - loss: 0.2099 - mae: 0.4802 - mse: 0.5744 - root_mean_squared_error: 0

17/17 [==============================] - 0s 919us/step - loss: 0.1992 - mae: 0.4648 - mse: 0.5404 - root_mean_squared_error: 0.7351
Epoch 449/500
17/17 [==============================] - 0s 919us/step - loss: 0.2081 - mae: 0.4780 - mse: 0.5701 - root_mean_squared_error: 0.7550
Epoch 450/500
17/17 [==============================] - 0s 919us/step - loss: 0.2079 - mae: 0.4775 - mse: 0.5696 - root_mean_squared_error: 0.7547
Epoch 451/500
17/17 [==============================] - 0s 919us/step - loss: 0.1935 - mae: 0.4556 - mse: 0.5292 - root_mean_squared_error: 0.7275
Epoch 452/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2002 - mae: 0.4674 - mse: 0.5453 - root_mean_squared_error: 0.7384
Epoch 453/500
17/17 [==============================] - 0s 919us/step - loss: 0.2003 - mae: 0.4670 - mse: 0.5457 - root_mean_squared_error: 0.7387
Epoch 454/500
17/17 [==============================] - 0s 919us/step - loss: 0.1945 - mae: 0.4587 - mse: 0.5250 - root_mean_squared_error: 0

Pipeline(steps=[('scaler', PowerTransformer()),
                ('estimator',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001B337780CA0>)])

In [34]:
model100 = grid_result.best_estimator_['estimator'].model

In [35]:
model100.save(r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\model-100-prec-1-10.h5')

In [36]:
dump(grid_result.best_estimator_['scaler'], r'C:\Users\Everton\Documents\Mestrado\projeto-final\experimentos\precipitacao-experimento-1\resultados\Experimento-10\scaler-100-prec-1-10.joblib') 

['C:\\Users\\Everton\\Documents\\Mestrado\\projeto-final\\experimentos\\precipitacao-experimento-1\\resultados\\Experimento-10\\scaler-100-prec-1-10.joblib']